In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
plt.rcParams['figure.figsize'] = [15, 10]
import collections, numpy
import glob
F = 200

In [ ]:
img_o = cv2.imread('inputs/P3/cart_1.png',0)
plt.imshow(img_o,cmap="gray")

ORGINAL_ITEMS = []
Prices = [40,83,6.5,30,5,50,8,108,6,139,144,36]
Name = ["Tuna-edible","Pril","Soda-drink","MihanMilk-drink","TinyCake-edible",
        "Toothpaste","MihanIceCreamCake-edible","Oil",
        "MihanIceCreamLivani-edible","Shampoo","Tea","Chips-edible"]
paths = glob.glob("./inputs/P3/items/*.png")
paths.sort()
for p in paths:
    tmp = cv2.imread(p,0).astype(np.float32)
    ORGINAL_ITEMS.append(tmp)

def imshow(img,ti="",show=True):
    plt.imshow(img,cmap="gray")
    plt.axis("off")
    if ti != "":
        plt.title(ti)
    if show:
        plt.show()
imshow(img_o,"Gray Scale of input image")


### #a

In [ ]:
img = img_o[75:,:]
img = img[:1075,:]
im_bw = cv2.threshold(img, 40, 255, cv2.THRESH_BINARY)[1]
imshow(img,"Border cropped")
imshow(im_bw,"Convert to binary")

def second_derivative_oc_de(img,s, ite=1):
    img_dilate = cv2.morphologyEx(img.astype(np.float32), cv2.MORPH_DILATE, np.ones((s, s), np.float32), iterations=ite)
    img_erode = cv2.morphologyEx(img.astype(np.float32), cv2.MORPH_ERODE, np.ones((s, s), np.float32), iterations=ite)
    img_open = cv2.morphologyEx(img.astype(np.float32), cv2.MORPH_OPEN, np.ones((s, s), np.float32), iterations=ite)
    img_close = cv2.morphologyEx(img.astype(np.float32), cv2.MORPH_CLOSE, np.ones((s, s), np.float32), iterations=ite)
    img_2nd = (img_close + img_open - img_dilate - img_erode) / 2
    return img_2nd

im_bw = second_derivative_oc_de(im_bw,11,1)
im_bw = cv2.threshold(im_bw, 40, 255, cv2.THRESH_BINARY)[1]
im_bw = im_bw.astype(np.uint8)
imshow(im_bw,"Second derivative of image using opening/closing - dilate/erode & make it binary again")


def imshow_components(labels):
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
    labeled_img[label_hue==0] = 0
    return cv2.cvtColor(labeled_img, cv2.COLOR_BGR2GRAY)

def RegFiller(counter,lb):
    out = np.zeros_like(im_bw)
    for key in counter:
        if key == 0 :
            continue
        x,y = np.where(lb==key,)
        res = np.array(list(zip(y,x)))
        if len(x) < 1000 :
            continue
        cv2.fillPoly(out, pts =[res], color=(255,255,255))
        
    return out

def ExtractItem (lb,key,img):
    out = np.zeros_like(im_bw)
    x,y = np.where(lb==key,)
    res = np.array(list(zip(y,x)))
    if len(x) < 1500 or key == 0 :
        return False,False
    cv2.fillPoly(out, pts =[res], color=(255,255,255))
    result = cv2.bitwise_and(img,img,out)
    xm,xM = min(x),max(x)
    ym,yM = min(y),max(y)
    return True,result[xm:xM,ym:yM]
    
    
    
num_labels, labels_im = cv2.connectedComponents(im_bw)
counter = collections.Counter(labels_im.flatten())
counter = dict(sorted(counter.items(), key=lambda item: item[1]))
filled = RegFiller(counter,labels_im)
imshow(filled,"Region filling based on connected components")


num_labels, labels_im = cv2.connectedComponents(filled)
counter = collections.Counter(labels_im.flatten())
counter = dict(sorted(counter.items(), key=lambda item: item[1]))

i = 1
Items = []
for key in counter:
    if key == 0 :
        continue
    state,res = ExtractItem(labels_im,key,img)
    if state :
        plt.subplot(3,6,i)
        imshow (res,"Item #{0}".format(i),False)
        Items.append(res)
        i+=1
        
plt.show()

In [ ]:
def Compare(img_1:np.array,img_2:np.array):
    img_1 = img_1.astype(np.uint8)
    img_2 = img_2.astype(np.uint8)
    img1_new = cv2.resize(img_1,img_2.shape[::-1],interpolation=cv2.INTER_CUBIC)
    orb = cv2.ORB_create(nfeatures=F)
    _, des1 = orb.detectAndCompute(img1_new, None)
    _, des2 = orb.detectAndCompute(img_2, None)


    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)

    return len(matches)
plt.rcParams['figure.figsize'] = [21, 14]

### #b

In [ ]:
def search(flag=None,_name=[]):
    counts = np.zeros(shape=(12,))
    _li = []
    _sco = []
    _org_index = []
    for index in range(len(Items)):
        score = -np.inf
        I = -1
        for i in range(12):
            _c = Compare(Items[index],ORGINAL_ITEMS[i])
            if _c > score :
                score = _c 
                I = i 
        
        counts[I] += 1 
        if flag == I:
            _li.append(index)
            _sco.append(score)
            _org_index.append(I)
            
        if len(_name) != 0 :
            for n in _name :
                if n in Name[I]:
                    _li.append(index)
                    _sco.append(score)
                    _org_index.append(I)
        
    if flag is not None :
        return _li,_sco,_org_index
    
    return counts 

plt.rcParams['figure.figsize'] = [12,8]

_indexes,_scorses,_ = search(-1,["Tiny"])
for i in range(len(_indexes)):
    plt.subplot(1,3,i+1)
    tmp = Items[_indexes[i]]
    name = Name[_indexes[i]]
    score = _scorses[i]
    imshow(tmp,"Recognized as Tiny with score: {0}/{1}".format(score,F),False)
plt.tight_layout()
plt.show()
    

### #c

In [ ]:
plt.rcParams['figure.figsize'] = [12,6]
_indexes,_scorses,_org_index = search(-1,["drink"])

p = 0 
for i in range(len(_indexes)):
    plt.subplot(1,3,i+1)
    tmp = Items[_indexes[i]]
    name = Name[_org_index[i]].split("-")[0]
    score = _scorses[i]
    p+= Prices[_org_index[i]]
    imshow(tmp,"Recognized {2}, Total Cost: {3}KT".format(score,F,name,p),False)
    
plt.tight_layout()
plt.show()
    

### #d

In [ ]:

plt.rcParams['figure.figsize'] = [12,8]

_indexes,_scorses,_ = search(-1,["Mihan"])
cnt = 0
p = 0 
for i in range(len(_indexes)):
    plt.subplot(2,2,i+1)
    tmp = Items[_indexes[i]]
    name = Name[_[i]].split("-")[0]
    score = _scorses[i]
    p+= Prices[_[i]]*1000
    cnt +=1
    imshow(tmp,"Recognized {2}, Total Cost: {3}, cnt:{4}".format(score,F,name,p,cnt),False)
plt.tight_layout()
plt.show()

### #e

In [ ]:
plt.rcParams['figure.figsize'] = [26, 14]
_indexes,_scorses,_ = search(-1,Name)
cnt = 0
p = 0 
for i in range(len(_indexes)):
    plt.subplot(3,6,i+1)
    tmp = Items[_indexes[i]]
    
    name = Name[_[i]].split("-")[0]
    score = _scorses[i]
    p+= Prices[_[i]]
    cnt +=1
    imshow(tmp,"{2}:{5}KT, Total Price: {3}KT({4})".format(score,F,name,p,cnt,float(Prices[_[i]])),False)
plt.tight_layout()
plt.show()